In [1]:
import torch

In [2]:
# setup device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda')

In [4]:
import yaml
from types import SimpleNamespace

# load config
config_path = "/export/home/workspace/dreambooth/diffusers/configs/zerobooth.yaml"
config = yaml.load(open(config_path, "r"), Loader=yaml.FullLoader)
config = SimpleNamespace(**config)

In [5]:
from modeling_zerobooth import ZeroBooth

model = ZeroBooth(config=config.model)

Creating model
Resizing position embedding grid-size from %s to %s (16, 16) (28, 28)
_IncompatibleKeys(missing_keys=[], unexpected_keys=['visual.proj', 'visual.ln_post.weight', 'visual.ln_post.bias', 'visual.transformer.resblocks.23.attn.in_proj_weight', 'visual.transformer.resblocks.23.attn.in_proj_bias', 'visual.transformer.resblocks.23.attn.out_proj.weight', 'visual.transformer.resblocks.23.attn.out_proj.bias', 'visual.transformer.resblocks.23.ln_1.weight', 'visual.transformer.resblocks.23.ln_1.bias', 'visual.transformer.resblocks.23.mlp.c_fc.weight', 'visual.transformer.resblocks.23.mlp.c_fc.bias', 'visual.transformer.resblocks.23.mlp.c_proj.weight', 'visual.transformer.resblocks.23.mlp.c_proj.bias', 'visual.transformer.resblocks.23.ln_2.weight', 'visual.transformer.resblocks.23.ln_2.bias'])
freeze class_embedding
freeze positional_embedding
freeze conv1.weight
freeze ln_pre.weight
freeze ln_pre.bias
freeze transformer.resblocks.0.attn.in_proj_weight
freeze transformer.resblocks.0.

In [6]:
load_from = "/export/home/workspace/dreambooth/diffusers/output/pretrain-20221211-bs=3-nq=8-prenorm-norm_clip=all_tune=4/65000"
model.load_checkpoint(load_from)
model.to(device)
model.eval()


ZeroBooth(
  (blip): BLIP(
    (visual_encoder): VisionTransformer(
      (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): Sequential(
          (0): ResidualAttentionBlock(
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
            )
            (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
              (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
              (gelu): QuickGELU()
              (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
            )
            (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (drop_path): Identity()
          )
          (1): ResidualAttentionBlock(
            (attn): MultiheadAttention(
 

In [7]:
from PIL import Image

# load image
def load_image(path):
    img = Image.open(path).convert("RGB")
    return img

In [8]:
def display_img(img):
    w, h = img.size
    f = 368 / max(w, h)

    display(img.resize((int(w * f), int(h * f))))

In [9]:
subject = "dog"
prompt = "a dog swimming in the ocean"

image_path = "/export/home/workspace/dreambooth/diffusers/data/NationalGeographic_2731043_4x3.webp"
image = load_image(image_path)

In [10]:
config.image_size

392

In [11]:
from train_zerobooth import create_transforms

# create transforms
transforms = create_transforms(config)

inp_tsfm = transforms["inp_image_transform"]
txt_tsfm = transforms["text_transform"]

In [12]:
samples = {
    "input_images": inp_tsfm(image).unsqueeze(0).to(device),
    "class_names": [txt_tsfm(subject)],
    "prompt": [txt_tsfm(prompt)],
}

In [13]:
for i in range(10):
    output = model.generate(samples, seed=38768122, guidance_scale=10)
    # output = model.generate(samples, seed=387658222, guidance_scale=7.5)

    display(output[0])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-13-b4342af2ad1c>:2 in <module>                                                    │
│                                                                                                  │
│ /export/home/workspace/dreambooth/diffusers/modeling_zerobooth.py:196 in generate                │
│                                                                                                  │
│   193 │   │   scheduler = self.eval_noise_scheduler                                              │
│   194 │   │                                                                                      │
│   195 │   │   # 1. extract BLIP query features and proj to text space -> (bs, 32, 768)           │
│ ❱ 196 │   │   query_embeds = self.blip(image=input_image, text=text_input)                       │
│   197 │   │   query_embeds = self.proj_layer(query_embeds)                                       │
│   198 │   │   query_embeds = self.pool_layer(query_embeds.transpose(1, 2)).transpose(1, 2)       │
│   199                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1102 in _call_impl             │
│                                                                                                  │
│   1099 │   │   # this function, and just call forward.                                           │
│   1100 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1101 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1102 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1103 │   │   # Do not call functions when jit is used                                          │
│   1104 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1105 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /export/home/workspace/dreambooth/diffusers/BLIP2/models/blipv2_feature_extractor.py:91 in       │
│ forward                                                                                          │
│                                                                                                  │
│    88 │                                                                                          │
│    89 │   def forward(self, image, text):                                                        │
│    90 │   │                                                                                      │
│ ❱  91 │   │   image_embeds = self.visual_encoder(image)                                          │
│    92 │   │   image_embeds = self.ln_vision(image_embeds)                                        │
│    93 │   │                                                                                      │
│    94 │   │   text_tokens = self.tokenizer(                                                      │
│                                                                                                  │
│ /opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1102 in _call_impl             │
│                                                                                                  │
│   1099 │   │   # this function, and just call forward.                                           │
│   1100 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1101 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1102 │   │   │   return forward_call(*input, **kwargs)   